some observations    
===============

**vocabulary size**    
tweet tokenizer / no preprocessing = 313803

In [45]:
import pickle
import pandas as pd
import math
from collections import Counter
from nltk.tokenize import TweetTokenizer

In [15]:
df = pd.read_csv('tweets.csv', sep='\t', usecols=[1,4], names=['id', 'tweet'])

In [46]:
tweets = df['tweet'].tolist()

In [30]:
tokenizer = TweetTokenizer()

In [40]:
#@ Brandon: this takes a while use pickled version instead (below)
#V = set()
#for tweet in tweets:
#    [V.add(token) for token in tokenizer.tokenize(tweet)]

#TODO: we can basically reuse our old code here which cleans and tokenized the tweets

In [42]:
len(V)

313803

In [44]:
#dump
with open('vocabulary_naive.pickle', 'wb') as f:
    pickle.dump(V, f)

In [ ]:
#load
with open('vocabulary_naive.pickle', 'rb') as f:
    V = pickle.load(f)

In [122]:
def tfidf(doc1, doc2, V, tweets):
    """
    d1,d2 -> list of tokens
    """
    #i think we can use this intersection here because words that are in the in one 
    #vector but not in the other result in zero multiplication
    intersect = set(doc1).intersection(set(doc2))
    c1 = Counter(doc1)#token count
    c2 = Counter(doc2)#token count
    d1 = {}
    d2 = {}
    for t in doc1:
        if t not in d1.keys() and t in intersect:
            #tf idf
            tf = c1[t]
            df = len([tweet for tweet in tweets if t in tweet]) #naive string matching
            idf = len(V) / df
            tfidf = (1 + math.log10(tf)) * (math.log10(idf))
            d1[t] = tfidf
    for t in doc2:
        if t not in d2.keys() and t in intersect:
            #tf idf
            tf = c2[t]
            
            #naive string matching
            #TODO: after tokenization and preprocessing was improved 
            #we should look if the token is in a tweet by comparing single tokens and not 
            #the whole string
            df = len([tweet for tweet in tweets if t in tweet])
            idf = len(V) / df
            tfidf = (1 + math.log10(tf)) * (math.log10(idf))
            d2[t] = tfidf
            df_tfidf = pd.DataFrame().from_dict(d1, orient='index')
            df_tfidf[1] = pd.DataFrame().from_dict(d2, orient='index')
    return df_tfidf

In [146]:
#@Brandon: its fun to play around with the those documents
doc1 = "this is a random tweet Hausarzt Affe Affe Affe".split()
doc2 = "this is random a a a a a tweet Hausarzt Hausarzt Hausarzt Hausarzt Hausarzt I think bla foo Affe".split()

In [147]:
df_tfidf = tfidf(doc1, doc2, V, tweets)
df_tfidf

,0,1
this,1.771972,1.771972
is,0.716318,0.716318
a,0.431264,0.732705
random,3.369552,3.369552
tweet,2.472582,2.472582
Hausarzt,4.154234,7.057920
Affe,5.441336,3.683744


In [139]:
def cosine(vec1, vec2):
    nominator = 0
    denominator = 0
    vec1_length = 0
    vec2_length = 0
    for v1,v2 in zip(vec1,vec2):
        nominator += v1*v2
        vec1_length += v1*v1
        vec2_length += v2*v2
    vec1_length = math.sqrt(vec1_length)
    vec2_length = math.sqrt(vec2_length)
    denominator = vec1_length * vec2_length 
    return nominator / denominator

In [148]:
cosine(df_tfidf[0], df_tfidf[1])

0.9298679480866374